In [4]:
%load_ext wurlitzer
#^^^ used to capture C/C++ output to be displayed in the notebook

import time
from klampt import *
from IPython.display import clear_output
import ipywidgets as widgets
import ipyklampt

#get the HOME/Klampt base directory
import os
KLAMPT_DIR = os.environ['HOME'] + '/Klampt'

world = WorldModel()
if world.loadFile(KLAMPT_DIR + "/data/tx90scenario0.xml"):
    sim = Simulator(world)
    kvis = ipyklampt.KlamptWidget(world)
    playback_widget = ipyklampt.Playback(kvis)
    
    #If you'd like to show the print output, comment out this line
    clear_output()
    display(playback_widget)
    display(kvis)
else:
    print "There was a problem loading the world file"

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

Playback(children=(Button(description=u'Play', icon=u'play', style=ButtonStyle(), tooltip=u'Start the animatio…

KlamptWidget(scene={u'object': {u'matrix': [1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 1], u'uuid': u'24g06…

Output()

In [2]:
#define your visualization and controller code here
import random

next_move_time = 2.0
move_delay = 5.0
robot = world.robot(0)

def init(world):
    global next_move_time
    kvis.add_text("HUD1",5,5,"0")
    next_move_time = 2.0
    
def control_loop(t,controller):
    global next_move_time
    kvis.add_text("HUD1",5,5,str(t))
    if t >= next_move_time:
        qmin,qmax = robot.getJointLimits()
        q = [random.uniform(a,b) for (a,b) in zip(qmin,qmax)]
        controller.setMilestone(q)
        next_move_time += move_delay
    pass


In [3]:
#this code runs the simulation and binds your code to the playback buttons
dt = 0.02

init(world)

def do_reset():
    sim.reset()
    sim.updateWorld()
    init(world)

def do_advance():
    global world,sim
    if world.numRobots() > 0:
        control_loop(sim.getTime(),sim.controller(0))  #call code in the above cell
    #print "Simulating...",dt
    sim.simulate(dt)
    sim.updateWorld()
    #print "Done."

#this binds the playback widget buttons
playback_widget.advance = do_advance
playback_widget.reset = do_reset
playback_widget.quiet = False